<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/PruebaControl_OCASO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Medición Prueba de Control
Archivo para medir las diferencias de accesos y lecturas en el grupo experimental frente al de control **(OJO, hay que añadir tb la parte de producción y tendencia).**

Diariamente se descargan los datos de la auditoria de los usuarios desde el lunes sábado 13 de Abril de 2024 a las 00:00 horas hasta el momento actual.

In [14]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.1 MB/s eta 0:00:00


In [80]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

Defino el grupo experimental como Gr1 y el de control como Gr2.

Hay que tener en cuenta que agentes_inicio son todos los agentes que había el viernes 12 de Abril, se hará un cruce con los agentes del grupo experimental para crear el Gr1.

**OJO: ¿QUÉ COLUMNAS ME QUEDO DE GR2 Y GR1 QUE VAYA A USAR DESPUÉS?**

##Archivos necesarios
Auditoria (descargar todos los días), estructura comercial. grupo control, estandar grupo experimental y formacion grupo experimental.

In [81]:
accesos_usuarios_df = pd.read_excel('/content/2.13_Auditoria_usuarios.xlsx')
agentes_inicio = pd.read_excel('/content/1.18_Estructura_Comercial.xlsx')
Gr2 = pd.read_excel('/content/1.18_Grupo_de_control.xlsx')
est_gr_exp = pd.read_excel('/content/estandar_gr_exp.xlsx')
for_gr_exp = pd.read_excel('/content/formacion_gr_exp.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [59]:
def clean_df(df):
    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]
    return df

def date_delete(df):
    df['DIA'] = df['FECHA ESPANOLA'].dt.day
    df['MES'] = df['FECHA ESPANOLA'].dt.month
    # Elimino la columna unnamed y la fecha del servidor, ya que nos vamos a quedar solo con la fecha española
    del_columns = ['UNNAMED: 0', 'FECHA SERVIDOR']
    df.drop(columns=del_columns, inplace=True)
    return df

In [82]:
accesos_usuarios_df = clean_df(accesos_usuarios_df)
accesos_usuarios_df = date_delete(accesos_usuarios_df)
agentes_inicio = clean_df(agentes_inicio)
Gr2 = clean_df(Gr2)
for_gr_exp = clean_df(for_gr_exp)
est_gr_exp = clean_df(est_gr_exp)

In [83]:
gr_exp = pd.concat([for_gr_exp, est_gr_exp], ignore_index=True)

In [79]:
Gr2

,RE,REGIONAL,SU,SUCURSAL,OFICINA,PV,CODIGO,COD. CALLIDUS/SOLNET,LDAP,NIF,...,NIF INSPECTOR,N. INSPECTOR,T. INSP,C. INSP CAPTADOR,NIF INSP CAPTADOR,N. INSP CAPTADOR,T. INSP CAPTADOR,S.RAPPEL,IRPF ESPECIAL,AUTOLIQUIDACION
0,-,-,-,-,-,946,2,44366436B0000,44366436B0000,44366436B,...,NaN,,NaN,NaN,NaN,,NaN,NO,NaN,NO
1,-,-,-,-,-,946,4,53568651H0000,53568651H0000,53568651H,...,NaN,,NaN,NaN,NaN,,NaN,NO,NaN,NO
2,-,-,-,-,-,946,5,043154084N0000,043154084N0000,043154084N,...,NaN,,NaN,NaN,NaN,,NaN,NO,NaN,NO
3,-,-,-,-,-,946,6,07229812S0000,07229812S0000,07229812S,...,NaN,,NaN,NaN,NaN,,NaN,NO,NaN,NO
4,-,-,-,-,-,946,8,9028697R0000,9028697R0000,9028697R,...,NaN,,NaN,NaN,NaN,,NaN,NO,NaN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20244,0595,0595 - ARAGON,0895,0895 - ZARAGOZA-COSO,0895 - ZARAGOZA-COSO,895,4523,73475305B0000,73475305B0000,73475305B,...,25445335K,ANA ISABEL AGLIO SANCHEZ,61.0,895004412.0,25445335K,ANA ISABEL AGLIO SANCHEZ,61.0,NO,NaN,NO
20245,0595,0595 - ARAGON,0895,0895 - ZARAGOZA-COSO,0895 - ZARAGOZA-COSO,895,4524,X8355768Y0000,X8355768Y0000,X8355768Y,...,25445335K,ANA ISABEL AGLIO SANCHEZ,61.0,895004412.0,25445335K,ANA ISABEL AGLIO SANCHEZ,61.0,NO,NaN,NO
20246,0595,0595 - ARAGON,0895,0895 - ZARAGOZA-COSO,0895 - ZARAGOZA-COSO,895,4525,72983780L0000,72983780L0000,72983780L,...,72967880N,CARLOS ALBERTO GINES CASTILLO,62.0,895004142.0,72967880N,CARLOS ALBERTO GINES CASTILLO,62.0,NO,NaN,NO
20247,0595,0595 - ARAGON,0895,0895 - ZARAGOZA-COSO,0895 - ZARAGOZA-COSO,895,4849,21807903Q0000,OQHO,21807903Q,...,21807903Q,CARLOS LUIS LEON RODAS,79.0,NaN,NaN,,NaN,NO,NaN,NO


In [84]:
Gr1 = agentes_inicio[agentes_inicio['COD. CALLIDUS/SOLNET'].isin(gr_exp['NAME'])]

La columna 'LDAP' es el código que identifica al agente en los archivos Gr1 y Gr2, y 'USERID' en accesos_usuarios_df, los que hay que tener en cuenta al cruzar los dataframes.

In [85]:
accesos_gr_exp = accesos_usuarios_df[accesos_usuarios_df['USERID'].isin(Gr1['LDAP'])]
accesos_gr_ctrl = accesos_usuarios_df[accesos_usuarios_df['USERID'].isin(Gr2['LDAP'])]

In [87]:
accesos_gr_exp

,USERID,NOMBRE USUARIO,TIPO EVENTO,FECHA ESPANOLA,DIA,MES
8,46852835H,DANIEL LLAMAS,READ,2024-04-15 12:20:11,15,4
9,46852835H,DANIEL LLAMAS,READ,2024-04-15 12:20:08,15,4
10,46852835H,DANIEL LLAMAS,LOGIN,2024-04-15 12:20:07,15,4
11,46852835H,DANIEL LLAMAS,READ,2024-04-15 12:20:07,15,4
175,O57R,MARTA VEGA,LOGOUT,2024-04-15 12:16:16,15,4
...,...,...,...,...,...,...
7827,O57R,MARTA VEGA,READ,2024-04-15 08:28:29,15,4
7828,O57R,MARTA VEGA,READ,2024-04-15 08:28:27,15,4
7829,O57R,MARTA VEGA,READ,2024-04-15 08:28:26,15,4
7830,O57R,MARTA VEGA,LOGIN,2024-04-15 08:28:25,15,4
